In [4]:
import scanpy as sc
import pandas as pd
import numpy as np
import kaleido


Bad key "text.kerning_factor" on line 4 in
/home/marta.sallese/miniconda3/envs/scvi_plots/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [1]:
import matplotlib

In [2]:
import plotly.express as px

In [5]:
sc.logging.print_versions()

-----
anndata     0.8.0
scanpy      1.9.2
-----
PIL                         9.4.0
anyio                       NA
asttokens                   NA
attr                        22.2.0
babel                       2.11.0
backcall                    0.2.0
beta_ufunc                  NA
binom_ufunc                 NA
brotli                      NA
certifi                     2023.07.22
cffi                        1.15.0
charset_normalizer          2.1.1
colorama                    0.4.6
comm                        0.1.2
cycler                      0.10.0
cython_runtime              NA
dateutil                    2.8.2
debugpy                     1.5.1
decorator                   5.1.1
defusedxml                  0.7.1
entrypoints                 0.4
executing                   1.2.0
fastjsonschema              NA
google                      NA
h5py                        3.6.0
hypergeom_ufunc             NA
idna                        3.4
igraph                      0.10.4
ipykernel            

In [6]:
adata = sc.read('/group/testa/Project/OvarianAtlas/Integrated_data_hdg_new/seacells_cancer_hdg_patients_batch_corr_scgen_tissuetreat.h5ad')

In [7]:
adata

AnnData object with n_obs × n_vars = 8735 × 17054
    obs: 'tissue', 'developmental_stage', 'treatment', 'recurrence', 'tumor_stage', 'paper_ID', 'anatomical_location', 'dataset', 'SEACell', '# Single Cells', 'tissue-treatment', '_scvi_batch', '_scvi_labels', 'concat_batch'
    uns: '_scvi_manager_uuid', '_scvi_uuid'
    obsm: 'corrected_latent', 'latent'

In [8]:
adata.obs = adata.obs.drop(columns=['_scvi_batch', '_scvi_labels', 'concat_batch'])

In [9]:
hvg = pd.read_csv('/home/marta.sallese/ov_cancer_atlas/Atlas_scripts/HVG/atlas_cancer_hdg_dispersion_patients.csv',  index_col=0)

In [10]:
hvg[hvg.highly_variable]

,highly_variable
FAM87B,True
SAMD11,True
PLEKHN1,True
HES4,True
ISG15,True
...,...
MT-ND3,True
MT-ND4L,True
MT-ND5,True
MT-ND6,True


In [11]:
hvg.highly_variable.sum()

5028

In [12]:
adata.var

""
FAM87B
LINC00115
FAM41C
SAMD11
NOC2L
...
MT-ND4L
MT-ND4
MT-ND5
MT-ND6


In [13]:
adata.var['highly_variable']=hvg.highly_variable

In [14]:
adata.var

,highly_variable
FAM87B,True
LINC00115,NaN
FAM41C,NaN
SAMD11,True
NOC2L,NaN
...,...
MT-ND4L,True
MT-ND4,NaN
MT-ND5,True
MT-ND6,True


In [15]:
adata.var.highly_variable = adata.var.highly_variable.fillna(False)

In [16]:
adata.var

,highly_variable
FAM87B,True
LINC00115,False
FAM41C,False
SAMD11,True
NOC2L,False
...,...
MT-ND4L,True
MT-ND4,False
MT-ND5,True
MT-ND6,True


In [17]:
# you can skip this passage if you specify use_highly_variable = True when computing embeddings
# adata.raw = adata
# adata = adata[:, adata.var.highly_variable]

In [18]:
adata.var_names

Index(['FAM87B', 'LINC00115', 'FAM41C', 'SAMD11', 'NOC2L', 'KLHL17', 'PLEKHN1',
       'HES4', 'ISG15', 'AGRN',
       ...
       'MT-CO2', 'MT-ATP8', 'MT-ATP6', 'MT-CO3', 'MT-ND3', 'MT-ND4L', 'MT-ND4',
       'MT-ND5', 'MT-ND6', 'MT-CYB'],
      dtype='object', length=17054)

In [19]:
cell_cycle_genes = [x.strip() for x in open('/home/marta.sallese/ov_cancer_atlas/regev_lab_cell_cycle_genes.txt')]

s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
cell_cycle_genes = [x for x in cell_cycle_genes if x in adata.var_names]

sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)

/home/marta.sallese/miniconda3/envs/scvi_plots/lib/python3.9/site-packages/scanpy/tools/_score_genes.py:151: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  for cut in np.unique(obs_cut.loc[gene_list]):
/home/marta.sallese/miniconda3/envs/scvi_plots/lib/python3.9/site-packages/scanpy/tools/_score_genes.py:151: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  for cut in np.unique(obs_cut.loc[gene_list]):


In [20]:
adata.obs

,tissue,developmental_stage,treatment,recurrence,tumor_stage,paper_ID,anatomical_location,dataset,SEACell,# Single Cells,tissue-treatment,S_score,G2M_score,phase
SEACell_patient_tissue,,,,,,,,,,,,,,
SEACell-5_Geistlinger_59_Metastasis,Metastasis,Advanced stage,CHT,Recurrence,IV,Geistlinger_59,Omentum,Geistlinger,SEACell-5,99,Metastasis_CHT,0.410004,0.408496,S
SEACell-7_Geistlinger_59_Metastasis,Metastasis,Advanced stage,CHT,Recurrence,IV,Geistlinger_59,Omentum,Geistlinger,SEACell-7,106,Metastasis_CHT,0.988536,1.013172,G2M
SEACell-2_Geistlinger_59_Metastasis,Metastasis,Advanced stage,CHT,Recurrence,IV,Geistlinger_59,Omentum,Geistlinger,SEACell-2,109,Metastasis_CHT,0.584550,0.331832,S
SEACell-8_Geistlinger_59_Metastasis,Metastasis,Advanced stage,CHT,Recurrence,IV,Geistlinger_59,Omentum,Geistlinger,SEACell-8,73,Metastasis_CHT,0.523011,0.375947,S
SEACell-6_Geistlinger_59_Metastasis,Metastasis,Advanced stage,CHT,Recurrence,IV,Geistlinger_59,Omentum,Geistlinger,SEACell-6,50,Metastasis_CHT,0.614094,0.672671,G2M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SEACell-88_Vasquez_118_Metastasis,Metastasis,Advanced stage,Naive,Unknown,IVB,Vasquez_118,Peritoneum,Vasquez,SEACell-88,9,Metastasis_Naive,-0.127419,-0.772315,G1
SEACell-67_Vasquez_118_Metastasis,Metastasis,Advanced stage,Naive,Unknown,IVB,Vasquez_118,Peritoneum,Vasquez,SEACell-67,7,Metastasis_Naive,-0.015085,-0.760213,G1
SEACell-32_Vasquez_118_Metastasis,Metastasis,Advanced stage,Naive,Unknown,IVB,Vasquez_118,Peritoneum,Vasquez,SEACell-32,4,Metastasis_Naive,0.065926,-0.537457,S


## I will now split the dataset by tissue type (Primary, Ascites, Metastasis) and compute the embeddings separately. Then on each of them I will perform leiden clustering and Sankey plots

### Primary 

In [21]:
adata_pr = adata[(adata.obs['tissue'] == 'Primary')]

In [26]:
adata_pr

AnnData object with n_obs × n_vars = 3750 × 17054
    obs: 'tissue', 'developmental_stage', 'treatment', 'recurrence', 'tumor_stage', 'paper_ID', 'anatomical_location', 'dataset', 'SEACell', '# Single Cells', 'tissue-treatment', 'S_score', 'G2M_score', 'phase'
    var: 'highly_variable'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'pca', 'neighbors', 'umap'
    obsm: 'corrected_latent', 'latent', 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [27]:
sc.tl.pca(adata_pr, use_highly_variable = True)

In [28]:
sc.pp.neighbors(adata_pr, n_neighbors=10, n_pcs=50)
sc.tl.umap(adata_pr)

In [29]:
leidenTotal=[]
for i in np.arange(0.1, 0.4, 0.03):
    sc.tl.leiden(adata_pr,resolution = i,key_added="leiden-{}".format(round(i,2)))
    leidenTotal.append("leiden-{}".format(round(i,2)))

In [30]:
leidenTotal

['leiden-0.1',
 'leiden-0.13',
 'leiden-0.16',
 'leiden-0.19',
 'leiden-0.22',
 'leiden-0.25',
 'leiden-0.28',
 'leiden-0.31',
 'leiden-0.34',
 'leiden-0.37',
 'leiden-0.4']

In [35]:
LeidenDF = adata_pr.obs[[i for i in adata_pr.obs.columns if "leiden-" in i]]

In [36]:
LeidenDF["color"] = LeidenDF["leiden-0.25"].astype(int)

/tmp/ipykernel_2106781/880194041.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LeidenDF["color"] = LeidenDF["leiden-0.25"].astype(int)


In [37]:
LeidenDF["treatment"] = adata_pr.obs["treatment"]
LeidenDF["treatment"] = LeidenDF["treatment"].replace({"Naive":0, "NACT":1, "CHT":2})

/tmp/ipykernel_2106781/928002017.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LeidenDF["treatment"] = adata_pr.obs["treatment"]
/tmp/ipykernel_2106781/928002017.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LeidenDF["treatment"] = LeidenDF["treatment"].replace({"Naive":0, "NACT":1, "CHT":2})


In [38]:
adata_pr.obs.tumor_stage.unique()

['IIIC', 'IIIB', 'IC', 'IIB', 'IVA', 'IVB']
Categories (6, object): ['IC', 'IIB', 'IIIB', 'IIIC', 'IVA', 'IVB']

In [39]:
LeidenDF["tumor_stage"] = adata_pr.obs["tumor_stage"]
LeidenDF["tumor_stage"] = LeidenDF["tumor_stage"].replace({"IIIC":0, "IIIB":1, "IC":2, "IIB":3, "IVA":4,"IVB":5})

/tmp/ipykernel_2106781/2914976512.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LeidenDF["tumor_stage"] = adata_pr.obs["tumor_stage"]
/tmp/ipykernel_2106781/2914976512.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LeidenDF["tumor_stage"] = LeidenDF["tumor_stage"].replace({"IIIC":0, "IIIB":1, "IC":2, "IIB":3, "IVA":4,"IVB":5})


In [40]:
adata_pr.obs.phase.unique()

['G2M', 'S', 'G1']
Categories (3, object): ['G1', 'G2M', 'S']

In [41]:
LeidenDF["phase"] = adata_pr.obs["phase"]
LeidenDF["phase"] = LeidenDF["phase"].replace({"G1":0, "G2M":1, "S":2})

/tmp/ipykernel_2106781/1336350546.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LeidenDF["phase"] = adata_pr.obs["phase"]
/tmp/ipykernel_2106781/1336350546.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LeidenDF["phase"] = LeidenDF["phase"].replace({"G1":0, "G2M":1, "S":2})


In [42]:
adata_pr.obs.recurrence.unique()

['Recurrence', 'Sensitive', 'Unknown']
Categories (3, object): ['Recurrence', 'Sensitive', 'Unknown']

In [43]:
LeidenDF["recurrence"] = adata_pr.obs["recurrence"]
LeidenDF["recurrence"] = LeidenDF["recurrence"].replace({"Recurrence":0, "Sensitive":1, "Unknown":2})

/tmp/ipykernel_2106781/1664838428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LeidenDF["recurrence"] = adata_pr.obs["recurrence"]
/tmp/ipykernel_2106781/1664838428.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LeidenDF["recurrence"] = LeidenDF["recurrence"].replace({"Recurrence":0, "Sensitive":1, "Unknown":2})


In [39]:
# LeidenDF[]

fig = px.parallel_categories(LeidenDF, 
                             dimensions=[i for i in adata_pr.obs.columns if "leiden-" in i],
                             color_continuous_scale=px.colors.sequential.Agsunset,color="color",
                             width=1000,height=700)
# fig.write_image('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_primary_leiden.png')
fig.write_html('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_primary_leiden.html')
# fig.show()

In [41]:
# LeidenDF[]

fig = px.parallel_categories(LeidenDF, 
                             dimensions=[i for i in adata_pr.obs.columns if "leiden-" in i],
                             color_continuous_scale=px.colors.sequential.Agsunset,color="treatment",
                             width=1000,height=700)
# fig.write_image('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_primary_treatment.png')
fig.write_html('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_primary_treatment.html')
# fig.show()

In [42]:
# LeidenDF[]

fig = px.parallel_categories(LeidenDF, 
                             dimensions=[i for i in adata_pr.obs.columns if "leiden-" in i],
                             color_continuous_scale=px.colors.sequential.Agsunset,color="phase",
                             width=1000,height=700)
# fig.write_image('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_primary_cellcycle.png')
fig.write_html('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_primary_cellcycle.html')
# fig.show()

In [44]:
# LeidenDF[]

fig = px.parallel_categories(LeidenDF, 
                             dimensions=[i for i in adata_pr.obs.columns if "leiden-" in i],
                             color_continuous_scale=px.colors.sequential.Agsunset,color="tumor_stage",
                             width=1000,height=700)
# fig.write_image('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_primary_tumorstage.png')
fig.write_html('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_primary_tumorstage.html')
# fig.show()

In [45]:
# LeidenDF[]

fig = px.parallel_categories(LeidenDF, 
                             dimensions=[i for i in adata_pr.obs.columns if "leiden-" in i],
                             color_continuous_scale=px.colors.sequential.Agsunset,color="recurrence",
                             width=1000,height=700)
# fig.write_image('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_primary_recurrence.png')
fig.write_html('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_primary_recurrence.html')
# fig.show()

In [ ]:
# selected leiden to see ontologies: 0.13, 0.25, 0.37
# final choice 0.25

### Ascites

In [69]:
adata_as = sc.read('/group/testa/Project/OvarianAtlas/Dea/Cancer_seacells/Ascites/adata_ascites_filtered_embeddings.h5ad')

In [70]:
adata_as

AnnData object with n_obs × n_vars = 772 × 17054
    obs: 'tissue', 'developmental_stage', 'treatment', 'recurrence', 'tumor_stage', 'paper_ID', 'anatomical_location', 'dataset', 'SEACell', '# Single Cells', 'tissue-treatment', 'S_score', 'G2M_score', 'phase', 'total_counts_seacell', 'leiden-0.1', 'leiden-0.2', 'leiden-0.3', 'leiden-0.4', 'leiden-0.5', 'leiden-0.6', 'leiden-0.7', 'leiden-0.8', 'leiden-0.9'
    var: 'highly_variable'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'leiden', 'leiden-0.1_colors', 'leiden-0.2_colors', 'leiden-0.3_colors', 'leiden-0.4_colors', 'leiden-0.5_colors', 'leiden-0.6_colors', 'leiden-0.7_colors', 'leiden-0.8_colors', 'leiden-0.9_colors', 'neighbors', 'pca', 'umap', 'wilcoxon_leiden-0.1', 'wilcoxon_leiden-0.2', 'wilcoxon_leiden-0.3', 'wilcoxon_leiden-0.4', 'wilcoxon_leiden-0.5', 'wilcoxon_leiden-0.6', 'wilcoxon_leiden-0.7', 'wilcoxon_leiden-0.8', 'wilcoxon_leiden-0.9'
    obsm: 'X_pca', 'X_umap', 'corrected_latent', 'latent'
    varm: 'PCs'
    obsp: '

In [71]:
LeidenDF = adata_as.obs[[i for i in adata_as.obs.columns if "leiden-" in i]]

In [72]:
LeidenDF["color"] = LeidenDF["leiden-0.5"].astype(int)

/tmp/ipykernel_2106781/995309661.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [73]:
LeidenDF["treatment"] = adata_as.obs["treatment"]

/tmp/ipykernel_2106781/2120274199.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [74]:
LeidenDF["treatment"] = LeidenDF["treatment"].replace({"Naive":0, "NACT":1, "CHT":2})

/tmp/ipykernel_2106781/1696152066.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [75]:
adata_as.obs.tumor_stage.unique()

['IIIC', 'IIIB', 'IVA', 'IVB']
Categories (4, object): ['IIIB', 'IIIC', 'IVA', 'IVB']

In [76]:
LeidenDF["tumor_stage"] = adata_as.obs["tumor_stage"]
LeidenDF["tumor_stage"] = LeidenDF["tumor_stage"].replace({"IIIB":0, "IIIC":1, "IVA":2,"IVB":3})

/tmp/ipykernel_2106781/2074490481.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_2106781/2074490481.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [77]:
adata_as.obs.phase.unique()

['G2M', 'G1', 'S']
Categories (3, object): ['G1', 'G2M', 'S']

In [78]:
LeidenDF["phase"] = adata_as.obs["phase"]
LeidenDF["phase"] = LeidenDF["phase"].replace({"G1":0, "G2M":1, "S":2})

/tmp/ipykernel_2106781/2418658085.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_2106781/2418658085.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [79]:
adata_as.obs.recurrence.unique()

['Recurrence', 'Unknown']
Categories (2, object): ['Recurrence', 'Unknown']

In [80]:
LeidenDF["recurrence"] = adata_as.obs["recurrence"]
LeidenDF["recurrence"] = LeidenDF["recurrence"].replace({"Recurrence":0, "Unknown":1})

/tmp/ipykernel_2106781/3599162756.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_2106781/3599162756.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [87]:
# LeidenDF[]

fig = px.parallel_categories(LeidenDF, 
                             dimensions=[i for i in adata_as.obs.columns if "leiden-" in i],
                             color_continuous_scale=px.colors.sequential.Agsunset,color="color",
                             width=1000,height=700)
#fig.write_image('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_ascites_leiden.png')
fig.write_html('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_ascites_leiden.html')
#fig.show()

In [82]:
# LeidenDF[]

fig = px.parallel_categories(LeidenDF, 
                             dimensions=[i for i in adata_as.obs.columns if "leiden-" in i],
                             color_continuous_scale=px.colors.sequential.Agsunset,color="treatment",
                             width=1000,height=700)
#fig.write_image('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_ascites_treatment.png')
fig.write_html('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_ascites_treatment.html')
# fig.show()

In [83]:
# LeidenDF[]

fig = px.parallel_categories(LeidenDF, 
                             dimensions=[i for i in adata_as.obs.columns if "leiden-" in i],
                             color_continuous_scale=px.colors.sequential.Agsunset,color="tumor_stage",
                             width=1000,height=700)
#fig.write_image('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_ascites_tumorstage.png')
fig.write_html('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_ascites_tumorstage.html')
# fig.show()

In [84]:
# LeidenDF[]

fig = px.parallel_categories(LeidenDF, 
                             dimensions=[i for i in adata_as.obs.columns if "leiden-" in i],
                             color_continuous_scale=px.colors.sequential.Agsunset,color="phase",
                             width=1000,height=700)
#fig.write_image('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_ascites_cellcycle.png')
fig.write_html('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_ascites_cellcycle.html')
# fig.show()

In [85]:
# LeidenDF[]

fig = px.parallel_categories(LeidenDF, 
                             dimensions=[i for i in adata_as.obs.columns if "leiden-" in i],
                             color_continuous_scale=px.colors.sequential.Agsunset,color="recurrence",
                             width=1000,height=700)
#fig.write_image('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_ascites_recurrence.png')
fig.write_html('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_ascites_recurrence.html')
# fig.show()

In [ ]:
# selected leiden to see ontologies
# chosen leiden 0.5

### Metastasis

In [45]:
adata_mt = adata[(adata.obs['tissue'] == 'Metastasis')]

In [46]:
adata_mt

View of AnnData object with n_obs × n_vars = 4004 × 17054
    obs: 'tissue', 'developmental_stage', 'treatment', 'recurrence', 'tumor_stage', 'paper_ID', 'anatomical_location', 'dataset', 'SEACell', '# Single Cells', 'tissue-treatment', 'S_score', 'G2M_score', 'phase'
    var: 'highly_variable'
    uns: '_scvi_manager_uuid', '_scvi_uuid'
    obsm: 'corrected_latent', 'latent'

In [47]:
sc.tl.pca(adata_mt, use_highly_variable = True)
sc.pp.neighbors(adata_mt, n_neighbors=10, n_pcs=50)
sc.tl.umap(adata_mt)

In [48]:
leidenTotal=[]
for i in np.arange(0.2, 0.4, 0.02):
    sc.tl.leiden(adata_mt,resolution = i,key_added="leiden-{}".format(round(i,2)))
    leidenTotal.append("leiden-{}".format(round(i,2)))

In [49]:
leidenTotal

['leiden-0.2',
 'leiden-0.22',
 'leiden-0.24',
 'leiden-0.26',
 'leiden-0.28',
 'leiden-0.3',
 'leiden-0.32',
 'leiden-0.34',
 'leiden-0.36',
 'leiden-0.38']

In [50]:
LeidenDF = adata_mt.obs[[i for i in adata_mt.obs.columns if "leiden-" in i]]

In [51]:
LeidenDF["color"] = LeidenDF["leiden-0.3"].astype(int)

/tmp/ipykernel_2106781/2077511126.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LeidenDF["color"] = LeidenDF["leiden-0.3"].astype(int)


In [52]:
LeidenDF["treatment"] = adata_mt.obs["treatment"]

/tmp/ipykernel_2106781/3208906183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LeidenDF["treatment"] = adata_mt.obs["treatment"]


In [53]:
LeidenDF["treatment"] = LeidenDF["treatment"].replace({"Naive":0, "NACT":1, "CHT":2})

/tmp/ipykernel_2106781/1696152066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LeidenDF["treatment"] = LeidenDF["treatment"].replace({"Naive":0, "NACT":1, "CHT":2})


In [54]:
adata_mt.obs.tumor_stage.unique()

['IV', 'IIIC', 'IIIB', 'IVB', 'IVA']
Categories (5, object): ['IIIB', 'IIIC', 'IV', 'IVA', 'IVB']

In [55]:
LeidenDF["tumor_stage"] = adata_mt.obs["tumor_stage"]
LeidenDF["tumor_stage"] = LeidenDF["tumor_stage"].replace({"IV":0, "IIIC":1, "IIIB":2, "IVB":3,"IVA":4})

/tmp/ipykernel_2106781/3174418450.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LeidenDF["tumor_stage"] = adata_mt.obs["tumor_stage"]
/tmp/ipykernel_2106781/3174418450.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LeidenDF["tumor_stage"] = LeidenDF["tumor_stage"].replace({"IV":0, "IIIC":1, "IIIB":2, "IVB":3,"IVA":4})


In [56]:
adata_mt.obs.phase.unique()

array(['S', 'G2M', 'G1'], dtype=object)

In [57]:
LeidenDF["phase"] = adata_mt.obs["phase"]
LeidenDF["phase"] = LeidenDF["phase"].replace({"G1":0, "G2M":1, "S":2})

/tmp/ipykernel_2106781/1582533869.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LeidenDF["phase"] = adata_mt.obs["phase"]
/tmp/ipykernel_2106781/1582533869.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LeidenDF["phase"] = LeidenDF["phase"].replace({"G1":0, "G2M":1, "S":2})


In [58]:
adata_mt.obs.recurrence.unique()

['Recurrence', 'Sensitive', 'Unknown']
Categories (3, object): ['Recurrence', 'Sensitive', 'Unknown']

In [59]:
LeidenDF["recurrence"] = adata_mt.obs["recurrence"]
LeidenDF["recurrence"] = LeidenDF["recurrence"].replace({"Recurrence":0, 'Sensitive':1, "Unknown":2})

/tmp/ipykernel_2106781/1440590878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LeidenDF["recurrence"] = adata_mt.obs["recurrence"]
/tmp/ipykernel_2106781/1440590878.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LeidenDF["recurrence"] = LeidenDF["recurrence"].replace({"Recurrence":0, 'Sensitive':1, "Unknown":2})


In [60]:
# LeidenDF[]

fig = px.parallel_categories(LeidenDF, 
                             dimensions=[i for i in adata_mt.obs.columns if "leiden-" in i],
                             color_continuous_scale=px.colors.sequential.Agsunset,color="color",
                             width=1000,height=700)
#fig.write_image('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_metastasis_leiden.png')
fig.write_html('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_metastasis_leiden.html')
# fig.show()

In [61]:
# LeidenDF[]

fig = px.parallel_categories(LeidenDF, 
                             dimensions=[i for i in adata_mt.obs.columns if "leiden-" in i],
                             color_continuous_scale=px.colors.sequential.Agsunset,color="treatment",
                             width=1000,height=700)
#fig.write_image('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_metastasis_treatment.png')
fig.write_html('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_metastasis_treatment.html')
# fig.show()

In [62]:
# LeidenDF[]

fig = px.parallel_categories(LeidenDF, 
                             dimensions=[i for i in adata_mt.obs.columns if "leiden-" in i],
                             color_continuous_scale=px.colors.sequential.Agsunset,color="phase",
                             width=1000,height=700)
#fig.write_image('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_metastasis_cellcycle.png')
fig.write_html('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_metastasis_cellcycle.html')
# fig.show()

In [63]:
# LeidenDF[]

fig = px.parallel_categories(LeidenDF, 
                             dimensions=[i for i in adata_mt.obs.columns if "leiden-" in i],
                             color_continuous_scale=px.colors.sequential.Agsunset,color="tumor_stage",
                             width=1000,height=700)
#fig.write_image('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_metastasis_tumorstage.png')
fig.write_html('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_metastasis_tumorstage.html')
# fig.show()

In [64]:
color_param = 'anatomical_location'

In [65]:
LeidenDF[color_param] = adata_mt.obs[color_param]

/tmp/ipykernel_2106781/2035870667.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [66]:
set(adata_mt.obs[color_param])

{'Bowel',
 'Mesentery',
 'Omentum',
 'Other',
 'Peritoneum',
 'Upper_quadrant',
 'Urinary_bladder'}

In [67]:
LeidenDF[color_param] = LeidenDF[color_param].replace({"Bowel":0, "Mesentery":1, "Omentum":2, "Other":3, "Peritoneum":4, "Tumor":5, "Upper_quadrant":6, "Urinary_bladder":7})

/tmp/ipykernel_2106781/2226268818.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [68]:
# LeidenDF[]

fig = px.parallel_categories(LeidenDF, 
                             dimensions=[i for i in adata_mt.obs.columns if "leiden-" in i],
                             color_continuous_scale=px.colors.sequential.Agsunset,color="anatomical_location",
                             width=1000,height=700)
#fig.write_image('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_metastasis_anatomy.png')
fig.write_html('/home/marta.sallese/ov_cancer_atlas/DEA/Sankeys/cancer_metastasis_anatomy.html')
# fig.show()

In [ ]:
# selected leiden to see ontologies
# final choice 0.11